# Transformações e Padronizações – Camada Trusted

Este notebook realiza a transformação dos dados provenientes da camada RAW, aplicando regras de limpeza, normalização, padronização de tipos, renomeação de colunas e modelagem estrutural.

In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DecimalType, IntegerType, LongType
from pyspark.sql import functions as f
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [2]:
builder: SparkSession.Builder = SparkSession.builder \
    .appName("Preparação TRS de Socios") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g")

In [3]:
spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/13 20:12:23 WARN Utils: Your hostname, wilcb, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/13 20:12:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/wilcb/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/wilcb/.ivy2.5.2/cache
The jars for the packages stored in: /home/wilcb/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-edbaa4e6-b9ca-4d4a-b5f1-a35d1005a27c;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 247ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [d

## Funções auxiliares

In [4]:
def null_count(df: DataFrame) -> None:
    """
    Conta e exibe a quantidade de valores nulos por coluna em um DataFrame do PySpark.

    A função percorre todas as colunas do DataFrame fornecido e calcula, para cada uma, 
    a quantidade de valores nulos (`NULL`). O resultado é exibido diretamente no console 
    por meio do método `.show()`.

    Args:
        df (DataFrame): DataFrame a ser inspecionado.

    Returns:
        None: A função apenas imprime o resultado no console.
    """
    try:
        # Cria uma lista de expressões que somam os valores nulos por coluna
        nulls = [
            f.sum(f.col(c).isNull().cast("int")).alias(c)
            for c in df.columns
        ]

        # Exibe os totais de nulos por coluna
        df.select(nulls).show()
    except Exception as e:
        print(f"[ERRO] Falha na busca de dados nulos: {e}")
        raise

## Leitura do DeltaTable na camada RAW

In [5]:

try:
    dt: DeltaTable = DeltaTable.forPath(spark, "../../RAW/socios/2025-12")
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

In [6]:
try:
    df: DataFrame = dt.toDF()
    df.show(truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na conversão para DataFrame: {e}")
    raise

26/01/13 20:12:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------------------+----------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+-------------------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social           |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|data_ingestao            |
+-----------+-------------------+----------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+-------------------------+
|07844030   |2                  |MARCIA APARECIDA CIRILO           |***335791**   |22                       |20060216              |NULL|***000000**        |NULL              |00                               |6           |2026-01-09 17:45:33.88378|


Contar valores nulos

In [7]:
null_count(df)

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+--------+-------------------+------------------+---------------------------------+------------+-------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|    pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|data_ingestao|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+--------+-------------------+------------------+---------------------------------+------------+-------------+
|          0|                  0|                   1832|         12831|                        0|                     0|26715378|                  0|          26115843|                                0|           0|            0|
+-----------+-------------------+-----------------------+--------------+----

In [8]:
df.select(df.data_entrada_sociedade).distinct().orderBy(f.col("data_entrada_sociedade").asc()).show()

+----------------------+
|data_entrada_sociedade|
+----------------------+
|              00080508|
|              00210823|
|              19000101|
|              19000508|
|              19000828|
|              19010101|
|              19111107|
|              19170131|
|              19170401|
|              19180817|
|              19190110|
|              19190530|
|              19191118|
|              19201020|
|              19220120|
|              19220330|
|              19221203|
|              19271109|
|              19290225|
|              19290416|
+----------------------+
only showing top 20 rows


In [9]:
df = df.withColumn(
    "data_entrada_sociedade",
    f.when(
        f.col("data_entrada_sociedade").isin("00080508", "00210823"),
        f.lit(None)
    ).otherwise(
        f.col("data_entrada_sociedade")
    )
)

df.select(df.data_entrada_sociedade).distinct().orderBy(f.col("data_entrada_sociedade").asc()).show()
df.select(df.data_entrada_sociedade).filter(df.data_entrada_sociedade.isNull()).count()

+----------------------+
|data_entrada_sociedade|
+----------------------+
|                  NULL|
|              19000101|
|              19000508|
|              19000828|
|              19010101|
|              19111107|
|              19170131|
|              19170401|
|              19180817|
|              19190110|
|              19190530|
|              19191118|
|              19201020|
|              19220120|
|              19220330|
|              19221203|
|              19271109|
|              19290225|
|              19290416|
|              19311008|
+----------------------+
only showing top 20 rows


2

## DROP COLUMN

Remoção de informações incompletas ou irrelevantes

In [10]:
df = df.drop("cnpj_cpf_socio")
df = df.drop("representante_legal")

## TIPOS DE DADOS

Ajuste nos tipos de dados

In [11]:
df.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- identificador_socio: string (nullable = true)
 |-- nome_socio_razao_social: string (nullable = true)
 |-- codigo_qualificacao_socio: string (nullable = true)
 |-- data_entrada_sociedade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- nome_representante: string (nullable = true)
 |-- codigo_qualificacao_representante: string (nullable = true)
 |-- faixa_etaria: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



In [12]:
df = df.withColumns({
    "identificador_socio": df.identificador_socio.cast(IntegerType()),
    "faixa_etaria": df.faixa_etaria.cast(IntegerType()),
    "codigo_qualificacao_socio": df.codigo_qualificacao_socio.cast(IntegerType()),
    "pais": df.pais.cast(IntegerType()),
    "codigo_qualificacao_representante": df.codigo_qualificacao_representante.cast(IntegerType()),
    "data_entrada_sociedade": f.to_date(df.data_entrada_sociedade, "yyyyMMdd")
})

In [13]:
df.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- identificador_socio: integer (nullable = true)
 |-- nome_socio_razao_social: string (nullable = true)
 |-- codigo_qualificacao_socio: integer (nullable = true)
 |-- data_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- nome_representante: string (nullable = true)
 |-- codigo_qualificacao_representante: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



## CONSULTAS / FILTROS

Quantidade de linhas de dados

In [14]:
print(f"Total: {df.count()}")

Total: 26806352


In [15]:
df.show(truncate=False)

+-----------+-------------------+----------------------------------+-------------------------+----------------------+----+------------------+---------------------------------+------------+-------------------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social           |codigo_qualificacao_socio|data_entrada_sociedade|pais|nome_representante|codigo_qualificacao_representante|faixa_etaria|data_ingestao            |
+-----------+-------------------+----------------------------------+-------------------------+----------------------+----+------------------+---------------------------------+------------+-------------------------+
|07844030   |2                  |MARCIA APARECIDA CIRILO           |22                       |2006-02-16            |NULL|NULL              |0                                |6           |2026-01-09 17:45:33.88378|
|07844099   |2                  |JOSE ELANDIO TIOTONIO FEITOSA     |49                       |2006-02-03            |NULL|NULL              

In [16]:
df.select(df.nome_representante).filter(df.nome_representante.isNotNull()).show(truncate=False)

print(f"Quantidade de representantes não nulos: {df.select(df.nome_representante).filter(df.nome_representante.isNotNull()).count()}")
print(f"Quantidade de representantes nulos: {df.select(df.nome_representante).filter(df.nome_representante.isNull()).count()}")


+-----------------------------------+
|nome_representante                 |
+-----------------------------------+
|EUGENIO RICARDO ARAUJO COSTA       |
|EUGENIO RICARDO ARAUJO COSTA       |
|EDWARD DE MATTOS VAZ               |
|AMADEU CRUZ BARBOSA FILHO          |
|AMADEU CRUZ BARBOSA FILHO          |
|SELENE DALCANALE                   |
|SIMONE SPERB GOMES                 |
|ROSANGELA SILVA E LUZ              |
|RONALDO VIEIRA BENTO               |
|PAULO ARMANDO BORN                 |
|PAULO ARMANDO BORN                 |
|CARLOS EDUARDO ESCOBAL             |
|CARLOS EDUARDO ESCOBAL             |
|JOAO MARCOS GIBRAM                 |
|MILTON CALHEIRO DOS SANTOS FILHO   |
|FRANCISCO EUSTACIO FERNANDES VIEIRA|
|SAMUEL GUERRA DA SILVA             |
|JESUS VILLARINO ALVES              |
|LAUDEVICO GOULART                  |
|SONIA MARIA MASSA RAMALHO          |
+-----------------------------------+
only showing top 20 rows
Quantidade de representantes não nulos: 690509
Quantidade de re

In [51]:
df.select(df.pais) \
    .filter(df.pais.isNotNull()).distinct() \
    .orderBy(f.col("pais").asc()).show(truncate=False)

print(f"Quantidade de código de paises não nulos: {df.select(df.pais).filter(df.pais.isNotNull()).count()}")
print(f"Quantidade de código de paises nulos: {df.select(df.pais).filter(df.pais.isNull()).count()}")

+----+
|pais|
+----+
|13  |
|15  |
|17  |
|23  |
|31  |
|37  |
|40  |
|41  |
|43  |
|47  |
|53  |
|59  |
|63  |
|64  |
|65  |
|69  |
|72  |
|73  |
|77  |
|80  |
+----+
only showing top 20 rows
Quantidade de código de paises não nulos: 90974
Quantidade de código de paises nulos: 26715378


Campo Faixa Etária:
>Baseada na data de nascimento do CPF de cada sócio, deverá ser criado o valor para o
>campo "Faixa Etária" conforme a regra abaixo:
- 1 para os intervalos entre 0 a 12 anos;
- 2 para os intervalos entre 13 a 20 anos;
- 3 para os intervalos entre 21 a 30 anos;
- 4 para os intervalos entre 31 a 40 anos;
- 5 para os intervalos entre 41 a 50 anos;
- 6 para os intervalos entre 51 a 60 anos;
- 7 para os intervalos entre 61 a 70 anos;
- 8 para os intervalos entre 71 a 80 anos; 
- 9 para maiores de 80 anos.
- 0 para não se aplica.

### Quantidade de sócios por faixa etária

In [ ]:
map_faixa = f.create_map(
    f.lit(1), f.lit("0 a 12 anos"),
    f.lit(2), f.lit("13 a 20 anos"),
    f.lit(3), f.lit("21 a 30 anos"),
    f.lit(4), f.lit("31 a 40 anos"),
    f.lit(5), f.lit("41 a 50 anos"),
    f.lit(6), f.lit("51 a 60 anos"),
    f.lit(7), f.lit("61 a 70 anos"),
    f.lit(8), f.lit("71 a 90 anos"),
    f.lit(9), f.lit("maiores de 80 anos"),
)

df_qtd_por_faixa_etaria = (
    df
    .groupBy("faixa_etaria")
    .count()
    .withColumnRenamed("count", "Quantidade")
    .withColumn(
        "descricao",
        f.coalesce(map_faixa[f.col("faixa_etaria")], f.lit("Não se aplica"))
    )
)

In [47]:
df_qtd_por_faixa_etaria.select(
    "faixa_etaria", "descricao", "Quantidade"
).orderBy(f.col("faixa_etaria")).show()

+------------+------------------+----------+
|faixa_etaria|         descricao|Quantidade|
+------------+------------------+----------+
|           0|     Não se aplica|    678531|
|           1|       0 a 12 anos|     25743|
|           2|      13 a 20 anos|    138621|
|           3|      21 a 30 anos|   1830365|
|           4|      31 a 40 anos|   4770354|
|           5|      41 a 50 anos|   6562684|
|           6|      51 a 60 anos|   5410023|
|           7|      61 a 70 anos|   4129991|
|           8|      71 a 90 anos|   2157244|
|           9|maiores de 80 anos|   1102796|
+------------+------------------+----------+



### Sócios mais velhos

Acima dos 60 anos

In [67]:
df.select("*").filter(df.faixa_etaria>=7).show(truncate=False)
print(f"Total: {df.select("*").filter(df.faixa_etaria>=7).count()}")

+-----------+-------------------+------------------------------------+-------------------------+----------------------+----+------------------+---------------------------------+------------+-------------------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social             |codigo_qualificacao_socio|data_entrada_sociedade|pais|nome_representante|codigo_qualificacao_representante|faixa_etaria|data_ingestao            |
+-----------+-------------------+------------------------------------+-------------------------+----------------------+----+------------------+---------------------------------+------------+-------------------------+
|04727343   |2                  |SHYDNEY JORGE ROSA                  |49                       |1998-04-03            |NULL|NULL              |0                                |7           |2026-01-09 17:45:33.88378|
|04727430   |2                  |HELIO TOLLINI                       |49                       |2001-10-08            |NULL|NULL    

### Empresas com sócio e representante

In [69]:
df.select("*").filter(df.nome_representante.isNotNull()).show(truncate=False)
print(f"Total: {df.select("*").filter(df.nome_representante.isNotNull()).count()}")

+-----------+-------------------+---------------------------------------------------------+-------------------------+----------------------+----+-----------------------------------+---------------------------------+------------+-------------------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social                                  |codigo_qualificacao_socio|data_entrada_sociedade|pais|nome_representante                 |codigo_qualificacao_representante|faixa_etaria|data_ingestao            |
+-----------+-------------------+---------------------------------------------------------+-------------------------+----------------------+----+-----------------------------------+---------------------------------+------------+-------------------------+
|15947450   |1                  |WLM PARTICIPACOES E COMERCIO DE MAQUINAS E VEICULOS S.A. |22                       |2009-10-26            |NULL|EUGENIO RICARDO ARAUJO COSTA       |5                                |0           |2026-01

### Média de sócios por empresa

In [54]:
df_media_socios = df.groupBy("cnpj_basico") \
    .count() \
    .select(f.round(f.avg("count"), 2).alias("media_socios")) \

df_media_socios.show()

+------------+
|media_socios|
+------------+
|         1.7|
+------------+



In [ ]:
null_count(df)

+-----------+-------------------+-----------------------+-------------------------+----------------------+--------+------------------+---------------------------------+------------+-------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|codigo_qualificacao_socio|data_entrada_sociedade|    pais|nome_representante|codigo_qualificacao_representante|faixa_etaria|data_ingestao|
+-----------+-------------------+-----------------------+-------------------------+----------------------+--------+------------------+---------------------------------+------------+-------------+
|          0|                  0|                   1832|                        0|                     2|26715378|          26115843|                                0|           0|            0|
+-----------+-------------------+-----------------------+-------------------------+----------------------+--------+------------------+---------------------------------+------------+-------------+

